# numpy
Could use numpy arrays of z3 objects to reflect the behavior of operations. Almost silly.


# Worlds / Universes
If I encapsulate my kernel state into worlds or if we go multiprocess,

We can talk about worlds. An inutitionsatic world, recfunction world. A world that can quote some other world.







In [29]:
x,y = smt.Ints("x y")
x1,y1 = smt.Reals("x y")
smt.deserialize(smt.And(x1 == 0, x + y == 0).serialize())

Z3Exception: b'(error "line 5 column 19: ambiguous constant reference, more than one constant with the same sort, use a qualified expression (as <symbol> <sort>) to disambiguate x")\n'

# quote
sexpr

https://en.wikipedia.org/wiki/Cons  https://en.wikipedia.org/wiki/S-expression

In [20]:
from kdrag.all import *

Sexp = kd.Inductive("Sexp")
Sexp.declare("Atom", ("atom", smt.StringSort()))
Sexp.declare("Pair", ("fst", Sexp), ("snd", Sexp)) # Cons (car, cdr). Recursing through List or Seq is not easy.
Sexp.declare("Nil")
Sexp.create()

# could even quote types
#def quote(x : smt.AstRef):

def cons_list(*args):
    acc = Sexp.Nil()
    for x in reversed(args):
        acc = Sexp.Pair(x, acc)
    return acc

def quote1(x : smt.AstRef):
    return smt.StringVal(x.srepr()) # x.serialize()?

def quote2(x : smt.ExprRef):
    if isinstance(x ,smt.QuantifierRef):
        raise Exception("Cannot quote quantifiers yet")
    elif isinstance(x, smt.IntNumRef):
    elif isinstance(x, smt.RatNumRef):
    elif isinstance(x, smt.AlgebraicNumRef):
    elif isinstance(x, smt.FPNumRef): 
    elif isinstance(x, smt.BitVecNumRef):
    elif smt.is_app(x):
        decl = x.decl()
        sdecl = Sexp.Atom(decl.name())
        args = [quote2(x) for x in x.children()]
        return cons_list(sdecl, *args)
    




AttributeError: 'Datatype' object has no attribute 'ctx_ref'

In [19]:
#IExpr
import functools

BoolRef = kd.Inductive("BoolRef")
BoolRef.declare("Const", ("name", smt.StringSort()))
BoolRef.declare("Abstract", ("abstract0", smt.StringSort())) # Could be sexp
BoolRef.declare("True_")
BoolRef.declare("False_")
BoolRef.declare("And", ("and0", BoolRef), ("and1", BoolRef))
BoolRef.declare("Or", ("or0", BoolRef), ("or1", BoolRef))
BoolRef.declare("Not", ("not0", BoolRef))
BoolRef.declare("Implies", ("impl0", BoolRef), ("impl1", BoolRef))
BoolRef.declare("Eq", ("eq0", BoolRef), ("eq1", BoolRef))
BoolRef = BoolRef.create()

def quote(x : smt.BoolRef) -> smt.DatatypeRef:
    assert isinstance(x, smt.BoolRef)
    if smt.is_true(x):
        return BoolRef.True_()
    elif smt.is_false(x):
        return BoolRef.False_()
    elif smt.is_and(x):
        return functools.reduce(BoolRef.And, map(quote, x.children()))
    elif smt.is_or(x):
        return functools.reduce(BoolRef.Or, map(quote, x.children()))
    elif smt.is_not(x):
        return BoolRef.Not(quote(x.arg(0)))
    elif smt.is_implies(x):
        return BoolRef.Implies(quote(x.arg(0)), quote(x.arg(1)))
    elif smt.is_eq(x):
        return BoolRef.Eq(quote(x.arg(0)), quote(x.arg(1)))
    elif smt.is_const(x):
        print(x)
        return BoolRef.Const(x.decl().name())
    else:
        return BoolRef.Abstract(x.srepr())


p,q,r = smt.Bools("p q r")
quote(p & q)

p


Z3Exception: Z3 expression expected

In [25]:
myand = smt.Function("and", smt.BoolSort(), smt.BoolSort())
myand == (p & q).decl()


False

In [ ]:
def is_defined_rec(x : smt.ExprRef) -> bool:
    """
    Traverse call graph to make sure either smt intrinsic or defined functions are reachable.
    """
    todo = [x.decl()]
    seen = {x.decl()}
    while todo:
        x = todo.pop()
        if smt.is_uninterp_decl(x) and x not in kd.kernel.defns:
            return False
        else:
            defn = kd.kernel.defns[x]
            unseen = [d for d in kd.utils.decls(defn.body) if d not in seen]
            seen.union(unseen)
            todo.extend(unseen)
    return True

        



Definition by observation
 I guess codefinition isn't really a thing?


 "q", ctx(a , b, q, x, y, z) ->  substitute_fun( , ctx, head) 

"q", [params], [ctx], kd.cond(
    (ctx == obs1, q) 
    ctx == obs2, 
    ctx == obs3
    )



kd.codefine("", obsevations?, body?)
kd.define_function
kd.define_cofunction

DoCatatype
CoInductive
field1(x)
field2(z)



# Generic Proof

check
schema
GenericSchema(GenericProof)
    callable
    def

# prolog auto
Use Lemma as a Prolog state.


# smart axiom
a higher level axiom.
a higher level define that does sanity checking. nitpicks termination



# search


In [4]:
from kdrag.all import *
import kdrag.theories.nat as nat
import kdrag.theories.real as real

# might be nice to do the join of multiple searchs. Contains add and mul for example
        if len(args) == 0:
            args = (self.top_goal().goal,)
        results = []
        for arg in args:
            results.append(kd.utils.search(arg, db=db))
        joinkey = min(len(r) for r in results)

x = smt.Const("x", nat.Nat)
search_expr(nat.Z + x + x)
z = smt.Real("z")
search_expr(real.sqrt(z))
search(real.sqrt)


[('kdrag.theories.real.sqrt_pos', |- ForAll(x, Implies(x >= 0, sqrt(x) >= 0))),
 ('kdrag.theories.real.sqrt_define', |- ForAll(x, sqrt(x) == x**(1/2))),
 ('kdrag.theories.real.sqrt_square',
  |- ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))),
 ('kdrag.theories.real.sqr_sqrt',
  |- ForAll(x, Implies(x >= 0, sqr(sqrt(x)) == x))),
 ('kdrag.theories.real.sqrt_sqr',
  |- ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x)))]

In [13]:
import kdrag as kd
import kdrag.smt as smt
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
l = kd.Lemma(smt.ForAll([n], nat.Z + n == n))
l.search()

{('kdrag.theories.nat.add.defn',
  |- ForAll([x, y],
         add(x, y) == If(is(Z, x), y, S(add(pred(x), y))))): [Z, Var(0)],
 ('kdrag.theories.nat.add_Z', |- ForAll(x, add(Z, x) == x)): [Var(0)],
 ('kdrag.theories.nat.add_comm',
  |- ForAll([x, y], add(x, y) == add(y, x))): [Z, Var(0)]}

In [2]:
smt.is_app(smt.Var(0, smt.IntSort()))

False

In [ ]:
        >>> import kdrag.theories.nat as nat
        >>> n = smt.Const("n", nat.Nat)
        >>> l = Lemma(smt.ForAll([n], nat.Z + n == n))
        >>> "kdrag.theories.nat.add_Z" in l.search().keys()
        >>> "kdrag.theories.nat.add_S" in l.search(nat.add).keys()

A search based tactic might work. Find instantiations. Add grounded into z3. Get unsat core. Prune with that.

blast_search(1) # Do it against goal only?
recusrively against right hand sides too? Hmm.



# finite sets
https://cvc5.github.io/docs/cvc5-1.0.2/theories/sets-and-relations.html#finite-sets

reflection to seqsort?
use wf?

There is actually a Card constraint.

https://docs.python.org/3/library/stdtypes.html#set

Fun.

image
preimage


topology.

If I can't tie the knot thorugh arrays,

Maybe I should take the definition based on the metric

https://github.com/Z3Prover/z3/discussions/7334
Hmm. I could get it to accept it through smtlib api...
I could parse smtlib.




In [ ]:
from z3 import *

MyType = Datatype('MyType')
MyType.declare('mk_nil')
MyTypeSort = MyType.create()
MyType.declare('mk_mytype', ('children', ArraySort(IntSort(), MyTypeSort)))
MyType = MyType.create()

x = Const("x", MyType)
s = Solver()
s.add(x != MyType.mk_nil)
print(s.check())
print(s.model())

In [34]:
MyType = smt.Datatype('MyType')
MyType.declare('mk_nil')
MyTypeSort = MyType.create()

MyType2 = smt.DeclareSort("MyType")
MyType2.eq(MyTypeSort) # False
MyType2
MyTypeSort

MyType

In [30]:
is_open(Set(R))
is_closed(Set(R))

is_open

is_open(A), is_open(B) => is_open(A intersect B)
is_closed(A), is_closed(B) => is_closed(A union B)

is_closed(A) => 

# finite sequences are closed
is_closed(Member(SeqA, A)) => is_closed(SeqSort(Set(R)))

is_open(Seq(A))

is_open(IntSort())

Indcutive("Open")
Basic(a,R,b, R)
CountUnion(R)
SeqInter(Seq(SeqA))


# Oh. But I can't tie the knot... Hmm.


SyntaxError: cannot assign to function call (2868927385.py, line 6)

In [ ]:

def range_(f: smt.ArraySortRef) -> smt.ArraySortRef:
    """
    Domain of a function.
    >>> IntSet = Set(smt.IntSort())
    >>> f = smt.Function("f", smt.IntSort(), smt.IntSort())
    >>> domain(f)
    domain(f)
    """
    x = smt.FreshConst(f.domain(), prefix="x")
    y = smt.FreshConst(f.range(), prefix="y")
    _ = Set(f.range())  # make sure that the appropriate overloads are registers
    return smt.Lambda([y], smt.Exists([x], f(x) == y))


def image(S: smt.ArraySort, f: smt.ArraySort):
    """
    Image of a set under a function.
    >>> IntSet = Set(smt.IntSort())
    >>> A = smt.Const("A", IntSet)
    >>> f = smt.Function("f", smt.IntSort(), smt.IntSort())
    >>> image(A, f)
    image(A, f)
    """

    return kd.QExists([y], member(y, S), f(y) == x)


def preimage(S: smt.ArraySort, f: smt.ArraySort):
    raise NotImplementedError("preimage")


In [ ]:
# let's us knmow what to scan. Also May enable us to traverse types more easily into their subpieces. List(T) sometimes we want to inspect T.
# We can do that by inspecting the dictionary
class GenericType():
    def __init__(self, builder):
        self.builder = builder
        self.memo = {}
    def __call__(self, *args, **kwds):
        key = (args, kwds)
        if key not in self.memo:
            T = self.builder(*args, **kwds)
            T._generic__ = self # a pointer back to ourselves
            self.memo[key] = self.builder(*args, **kwds)
        return self.memo[key]
    
 

In [ ]:
@functools.cache
def FinSet(T):
    kd.NewType(f"FinSet_{T.name()}", smt.SetSort(T), pred=lambda x: smt.Exists([n], smt.SetHasSize(x, n)))


smt.SetHasSize(a, k)


@functools.cache
def Set(T):
    S = smt.SetSort(T)
    kd.notation.or_.register(S, SetUnion)
    kd.notation.and_.register(S, SetIntersection)
    kd.notation.sub.register(S, SetDifference)
    kd.notation.le.register(S, IsSubset)
    kd.notation.lt.register(S, lambda x,y: IsSubset(x,y) & x != y)

    # induction principle?
    empty = smt.SetEmpty(T)
    full = smt.SetFull(T)

    

    comm_inter
    comm_union
    assoc_inter
    assoc_union
    idem_union
    idem_inter


def has_size(s, n):
    return smt.SetHasSize(s, n)

def remove(s, x):
    return smt.SetRemove(s, x)

def add(s, x):
    return smt.SetAdd(s, x)

def union(s1, s2):
    return smt.SetUnion(s1, s2)

def inter(s1, s2):
    return smt.SetIntersection(s1, s2)

def diff(s1, s2):
    return smt.SetDifference(s1, s2)

def is_subset(s1, s2):
    return smt.IsSubset(s1, s2)

def is_member(x, s):
    return smt.SetContains(s, x)

def empty(T):
    SetSort(T)
    return smt.SetEmpty(T)

def full(T)
    SetSort(T)
    return 




# Reflect
I could quote/reflect terms to DSLs rather than a giant SMT generic thing.
For example: polynomials, diffexpr, 


https://www-sop.inria.fr/members/Yves.Bertot/coqart-chapter16.pdf
Proof by reflection is a chartersitc feature of type theory? I don't think so. Oh, maybe he's referring to refl. Ok. This seems like a nice book

http://adam.chlipala.net/cpdt/html/Cpdt.Reflection.html


eval_(e, z3.__dict__) ought to return basically the original thing. (Exactly the original thing?)


In [ ]:
Expr = kd.Inductive("RExpr")
RExpr.declare("Const", ("value", smt.RealSort()))
RExpr.declare("X")
RExpr.declare("Add", ("add0", RExpr), ("add1", RExpr))
RExpr.declare("Mul", ("mul0", RExpr), ("mul1", RExpr))
RExpr.declare("Exp", ("exp0", RExpr))
RExpr = RExpr.create()

e = smt.Const("e", RExpr)
x = smt.Real("x")
RExpr_eval = smt.Function("RExpr_eval", RExpr, smt.ArraySort(smt.RealSort(), smt.RealSort()))
RExpr_eval = smt.Function("RExpr_eval", [e], Lambda([x],
    kd.cond(
        (e.is_Const, e.value),
        (e.is_X, x),
        (e.is_Add, RExpr_eval(e.add0) + RExpr_eval(e.add1)),
        (e.is_Mul, RExpr_eval(e.mul0) * RExpr_eval(e.mul1)),
        (e.is_Exp, real.exp(RExpr_eval(e.exp0)))
    )  
))
        

#DiffExpr.declare("Log", ("log0", DiffExpr))

def RExpr_reflect(v, e : smt.ArithRef) -> smt.DatatypeRef:
    if e == v:
        return RExpr.X
    elif smt.is_rational_value(e) or smt.is_algebraic_value():
        return RExpr.Const(e)
    else:
        i = constructor_of_name(RExpr, e.decl().name().upper())
        return RExpr.constructor(i, *[RExpr_reflect(arg) for arg in e.children()])

# This is a generic concept.
# Given any datatype with names that are the same as funcdecls
# we can generate a reflection function.
def reflect(dt : smt.DatatypeSortRef):
    pass
# or
def reflect_decls(*decls : smt.FuncDeclRef):
    pass
    # build a datatype corresponding to the decls and/or an eval_ function back into the interpretation and/or a meta reflection function.



# cvc5 dreal


In [4]:
from kdrag.all import *
import kdrag
import cvc5.pythonic as cvc5
import kdrag.reflect

def cvc5_sort(s : smt.SortRef):
    if s == smt.RealSort():
        return cvc5.RealSort()
    elif s == smt.IntSort():
        return cvc5.IntSort()
    elif s == smt.BoolSort():
        return cvc5.BoolSort()
    elif isinstance(s, smt.ArraySortRef):
        return cvc5.ArraySort(cvc5_sort(s.domain()), cvc5_sort(s.range()))
    else:
        raise NotImplementedError(f"Sort {s} not supported by cvc5 translator")

def default_cvc5(e):
    if smt.is_const(e):
        return cvc5.Const(e.decl().name(), cvc5_sort(e.sort()))
    elif smt.is_app(e):
        decl, args = e.decl(), e.children()
        print(decl.name(), *[cvc5_sort(arg.sort()) for arg in args], cvc5_sort(e.sort()))
        f = cvc5.Function(decl.name(), *[cvc5_sort(arg.sort()) for arg in args], cvc5_sort(e.sort()))
        return f(*map(to_cvc5,args))
    else:
        raise NotImplementedError(f"Expression {e} not supported by cvc5")

def to_cvc5(e):
    return kdrag.reflect.eval_(e, globals=cvc5.__dict__, default=default_cvc5)
import kdrag.theories.real as real
x,y = smt.Reals("x y")
cvc5.simplify(to_cvc5(real.cos(x)**2 + real.sin(x)**2))
#to_cvc5(x)
#cvc5.Const("x", cvc5.RealSort())

cos[x]*cos[x] + sin[x]*sin[x]

# sympy
Sympy can do sums also.
Convfirming those can be quite automatic.
Finding closed forms
Singularities





In [5]:
%load_ext line_profiler

In [10]:

def foo():
    import kdrag.smt as smt
    from kdrag.smt import ForAll, Function
    import kdrag as kd

    """
    Definitions about the reals.
    Transcendental functions and bounds.

    """
    R = smt.RealSort()
    RFun = smt.ArraySort(R, R)
    RSeq = smt.ArraySort(smt.IntSort(), R)

    real_db = []
    real_simp = []


    def rlemma(thm, by=[], **kwargs):
        return kd.prove(thm, by + real_db + real_simp, **kwargs)


    x, y, z = smt.Reals("x y z")

    f, g = smt.Consts("f g", RFun)
    fadd = kd.notation.add.define([f, g], smt.Lambda([x], f[x] + g[x]))
    fsub = kd.notation.sub.define([f, g], smt.Lambda([x], f[x] - g[x]))
    fmul = kd.notation.mul.define([f, g], smt.Lambda([x], f[x] * g[x]))
    fdiv = kd.notation.div.define([f, g], smt.Lambda([x], f[x] / g[x]))

    poly = smt.Function("poly", RFun, smt.BoolSort())
    kd.axiom(ForAll([x], poly(smt.K(smt.RealSort(), x))))
    kd.axiom(poly(smt.Lambda([x], x)))
    kd.axiom(kd.QForAll([f, g], poly(f), poly(g), poly(f + g)))
    kd.axiom(kd.QForAll([f, g], poly(f), poly(g), poly(f * g)))


    # NReal = kd.NewType("NReal", R)


    add = kd.define("add", [x, y], x + y)

    add_0 = kd.prove(ForAll([x], add(x, 0) == x), by=[add.defn])
    add_comm = kd.prove(ForAll([x, y], add(x, y) == add(y, x)), by=[add.defn])
    add_assoc = kd.prove(
        smt.ForAll([x, y, z], add(x, add(y, z)) == add(add(x, y), z)), by=[add.defn]
    )

    sub = kd.define("sub", [x, y], x - y)
    sub_0 = kd.prove(ForAll([x], sub(x, 0) == x), by=[sub.defn])
    sub_add = kd.prove(
        ForAll([x, y, z], (add(x, y) == z) == (x == sub(z, y))), by=[sub.defn, add.defn]
    )

    mul = kd.define("mul", [x, y], x * y)
    mul_zero = kd.prove(ForAll([x], mul(x, 0) == 0), by=[mul.defn])
    mul_1 = kd.prove(ForAll([x], mul(x, 1) == x), by=[mul.defn])
    mul_comm = kd.prove(ForAll([x, y], mul(x, y) == mul(y, x)), by=[mul.defn])
    # mul_assoc = kd.prove(
    #    ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)), by=[mul.defn]
    # )
    _l = kd.Lemma(smt.ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)))
    _x, _y, _z = _l.fixes()
    _l.unfold()
    _l.auto()
    mul_assoc = _l.qed()

    mul_distrib = kd.prove(
        ForAll([x, y, z], mul(x, add(y, z)) == add(mul(x, y), mul(x, z))),
        by=[mul.defn, add.defn],
    )


    def abstract_arith(t: smt.ExprRef) -> smt.ExprRef:
        """
        Z3 has difficult ematching over arithmetic expressions.
        Abstracting them to uninterpreted functions can help.
        """
        if smt.is_var(t):
            return t
        elif smt.is_app(t):
            if t.decl() == (x + y).decl():
                return add(abstract_arith(t.arg(0)), abstract_arith(t.arg(1)))
            elif t.decl() == (x - y).decl():
                return sub(abstract_arith(t.arg(0)), abstract_arith(t.arg(1)))
            elif t.decl() == (x * y).decl():
                return mul(abstract_arith(t.arg(0)), abstract_arith(t.arg(1)))
            else:
                f = t.decl()
                return f(*[abstract_arith(c) for c in t.children()])
        else:
            raise Exception("unimplemented case in abstract_arith", t)


    abs = kd.define("absR", [x], smt.If(x >= 0, x, -x))
    sgn = kd.define("sgn", [x], smt.If(x > 0, 1, smt.If(x < 0, -1, 0)))

    sgn_abs = kd.prove(ForAll([x], abs(x) * sgn(x) == x), by=[abs.defn, sgn.defn])
    abs_le = kd.prove(
        ForAll([x, y], (abs(x) <= y) == smt.And(-y <= x, x <= y)), by=[abs.defn]
    )
    abs_idem = kd.prove(ForAll([x], abs(abs(x)) == abs(x)), by=[abs.defn])
    abs_neg = kd.prove(ForAll([x], abs(-x) == abs(x)), by=[abs.defn])
    abs_pos = kd.prove(ForAll([x], abs(x) >= 0), by=[abs.defn])
    abs_add = kd.prove(ForAll([x, y], abs(x + y) <= abs(x) + abs(y)), by=[abs.defn])
    abs_mul = kd.prove(ForAll([x, y], abs(x * y) == abs(x) * abs(y)), by=[abs.defn])
    abs_triangle = kd.prove(
        ForAll([x, y, z], abs(x - y) <= abs(x - z) + abs(z - y)), by=[abs.defn]
    )


    nonneg = kd.define("nonneg", [x], abs(x) == x)
    nonneg_ge_0 = kd.prove(ForAll([x], nonneg(x) == (x >= 0)), by=[nonneg.defn, abs.defn])

    max = kd.define("max", [x, y], smt.If(x >= y, x, y))
    max_le = kd.prove(ForAll([x, y], (x <= y) == (max(x, y) == y)), by=[max.defn])
    max_comm = kd.prove(ForAll([x, y], max(x, y) == max(y, x)), by=[max.defn])
    max_assoc = kd.prove(
        ForAll([x, y, z], max(x, max(y, z)) == max(max(x, y), z)), by=[max.defn]
    )
    max_idem = kd.prove(ForAll([x], max(x, x) == x), by=[max.defn])
    max_ge = kd.prove(ForAll([x, y], max(x, y) >= x), by=[max.defn])
    max_ge_2 = kd.prove(ForAll([x, y], max(x, y) >= y), by=[max.defn])

    min = kd.define("min", [x, y], smt.If(x <= y, x, y))
    min_le = kd.prove(ForAll([x, y], (x <= y) == (min(x, y) == x)), by=[min.defn])
    min_comm = kd.prove(ForAll([x, y], min(x, y) == min(y, x)), by=[min.defn])
    min_assoc = kd.prove(
        ForAll([x, y, z], min(x, min(y, z)) == min(min(x, y), z)), by=[min.defn]
    )
    min_idem = kd.prove(ForAll([x], min(x, x) == x), by=[min.defn])
    min_le_2 = kd.prove(ForAll([x, y], min(x, y) <= x), by=[min.defn])
    min_le_3 = kd.prove(ForAll([x, y], min(x, y) <= y), by=[min.defn])


    n, m = smt.Ints("n m")
    to_real_add = kd.prove(
        ForAll([n, m], smt.ToReal(n + m) == smt.ToReal(n) + smt.ToReal(m))
    )
    to_real_sub = kd.prove(
        ForAll([n, m], smt.ToReal(n - m) == smt.ToReal(n) - smt.ToReal(m))
    )
    to_real_mul = kd.prove(
        ForAll([n, m], smt.ToReal(n * m) == smt.ToReal(n) * smt.ToReal(m))
    )
    to_real_neg = kd.prove(ForAll([n], smt.ToReal(-n) == -smt.ToReal(n)))
    to_real_inj = kd.prove(ForAll([n, m], (smt.ToReal(n) == smt.ToReal(m)) == (n == m)))
    to_real_mono = kd.prove(ForAll([n, m], (n < m) == (smt.ToReal(n) < smt.ToReal(m))))

    floor = kd.define("floor", [x], smt.ToReal(smt.ToInt(x)))
    n = smt.Int("n")
    int_real_galois_lt = kd.prove(ForAll([x, n], (x < smt.ToReal(n)) == (smt.ToInt(x) < n)))
    int_real_galois_le = kd.prove(
        ForAll([x, n], (smt.ToReal(n) <= x) == (n <= smt.ToInt(x)))
    )

    _2 = kd.prove(ForAll([x], smt.ToInt(floor(x)) == smt.ToInt(x)), by=[floor.defn])
    floor_idem = kd.prove(ForAll([x], floor(floor(x)) == floor(x)), by=[floor.defn, _2])
    floor_le = kd.prove(ForAll([x], floor(x) <= x), by=[floor.defn])
    floor_gt = kd.prove(ForAll([x], x < floor(x) + 1), by=[floor.defn])

    # c = kd.Calc([n, x], smt.ToReal(n) <= x)
    # c.eq(n <= smt.ToInt(x))
    # c.eq(smt.ToReal(n) <= floor(x), by=[floor.defn])
    # floor_minint = c.qed(defns=False)


    pow = kd.define("pow", [x, y], x**y)
    pow_add = kd.axiom(
        kd.QForAll([x, y, z], x >= 0, pow(x, y + z) == pow(x, y) * pow(x, z))
    )
    pow_one = kd.prove(kd.QForAll([x], pow(x, 1) == x), by=[pow.defn])
    pow_two = kd.prove(kd.QForAll([x], pow(x, 2) == x * x), by=[pow.defn])
    pow_three = kd.prove(kd.QForAll([x], pow(x, 3) == x * x * x), by=[pow.defn])
    # pow_zero = kd.kernel.prove(kd.QForAll([x], x > 0, pow(x, 0) == 1), by=[pow.defn])
    kd.kernel.prove(smt.Implies(x > 0, x**0 == 1))
    # pow_pos = kd.prove(kd.QForAll([x, y], x > 0, pow(x, y) > 0), by=[pow.defn])

    sqr = kd.define("sqr", [x], x * x)


    sqrt = kd.define("sqrt", [x], x**0.5)

    _l = kd.Lemma(kd.QForAll([x], x >= 0, sqrt(x) >= 0))
    _ = _l.fix()
    _l.unfold()
    _l.auto()
    sqrt_pos = _l.qed()

    sqrt_define = kd.prove(smt.ForAll([x], sqrt(x) == x**0.5), by=[sqrt.defn, pow.defn])

    _l = kd.Lemma(kd.QForAll([x], x >= 0, sqrt(x) ** 2 == x))
    _ = _l.fix()
    _l.unfold()
    _l.auto()
    sqrt_square = _l.qed()

    sqr_sqrt = kd.prove(
        kd.QForAll([x], x >= 0, sqr(sqrt(x)) == x), by=[sqrt_square, sqr.defn]
    )

    _l = kd.Lemma(kd.QForAll([x], x >= 0, sqrt(sqr(x)) == x))
    _ = _l.fix()
    _l.unfold()
    _l.auto()
    sqrt_sqr = _l.qed()

    exp = smt.Const("exp", kd.R >> kd.R)
    exp_add = kd.axiom(smt.ForAll([x, y], exp(x + y) == exp(x) * exp(y)))
    exp_lower = kd.axiom(
        smt.ForAll([x], exp(x) >= 1 + x)
    )  # tight low approximation at x = 0.
    exp_pos = kd.axiom(smt.ForAll([x], exp(x) > 0))  # maybe we can derive this one?

    exp_zero = kd.prove(smt.ForAll([x], exp(0) == 1), by=[exp_add, exp_pos])

    exp_div = kd.prove(smt.ForAll([x, y], exp(x) * exp(-x) == 1), by=[exp_add, exp_zero])
    exp_nzero = kd.prove(smt.ForAll([x], exp(x) != 0), by=[exp_div])
    exp_inv = kd.prove(smt.ForAll([x], exp(-x) == 1 / exp(x)), by=[exp_div])


    ln = smt.Function("ln", kd.R, kd.R)
    ln_exp = kd.axiom(smt.ForAll([x], ln(exp(x)) == x))
    # TODO. some of these are redundant depending on the range of ln being R.
    ln_mul = kd.axiom(kd.QForAll([x, y], x > 0, y > 0, ln(x * y) == ln(x) + ln(y)))
    ln_one = kd.prove(smt.ForAll([x], ln(1) == 0), by=[ln_exp, exp_zero])

    exp_ln = kd.axiom(kd.QForAll([x], x > 0, exp(ln(x)) == x))


    cos = smt.Const("cos", kd.R >> kd.R)
    sin = smt.Const("sin", kd.R >> kd.R)

    pythag = kd.axiom(smt.ForAll([x], cos(x) ** 2 + sin(x) ** 2 == 1))
    cos_abs_le = kd.prove(smt.ForAll([x], abs(cos(x)) <= 1), by=[pythag, abs.defn])
    sin_abs_le = kd.prove(smt.ForAll([x], abs(sin(x)) <= 1), by=[pythag, abs.defn])

    cos_0 = kd.axiom(cos(0) == 1)
    sin_0 = kd.prove(sin(0) == 0, by=[pythag, cos_0])

    pi = smt.Const("pi", kd.R)
    pi_bnd = kd.axiom(smt.And(3.14159 < pi, pi < 3.14160))

    cos_pi = kd.axiom(cos(pi) == -1)
    sin_pi = kd.prove(sin(pi) == 0, by=[pythag, cos_pi])

    cos_neg = kd.axiom(smt.ForAll([x], cos(-x) == cos(x)))
    sin_neg = kd.axiom(smt.ForAll([x], sin(-x) == -sin(x)))

    cos_add = kd.axiom(smt.ForAll([x, y], cos(x + y) == cos(x) * cos(y) - sin(x) * sin(y)))
    sin_add = kd.axiom(smt.ForAll([x, y], sin(x + y) == sin(x) * cos(y) + cos(x) * sin(y)))

    tan = kd.define("tan", [x], sin(x) / cos(x))
    atan = smt.Const("atan", kd.R >> kd.R)


    comp = kd.define("comp", [f, g], smt.Lambda([x], f(g(x))))
    kd.notation.matmul.register(RFun, comp)

    EReal = smt.Datatype("EReal")
    EReal.declare("real", ("val", smt.RealSort()))
    EReal.declare("inf")
    EReal.declare("neg_inf")
    EReal.declare("NaN")
    EReal = EReal.create()

    EPosReal = smt.Datatype("EPosReal")
    EPosReal.declare("real", ("val", smt.RealSort()))
    EPosReal.declare("inf")
    EPosReal = EPosReal.create()
    x_p = smt.Const("x", EPosReal)
    kd.notation.wf.define([x_p], smt.Implies(x_p.is_real, x_p.val >= 0))

    ident = kd.define("ident", [], smt.Lambda([x], x))
    const = kd.define("const", [x], smt.K(smt.RealSort(), x))
    X = ident

    # https://en.wikipedia.org/wiki/Cauchy_sequence
    a = smt.Const("a", RSeq)
    mod = smt.Const("mod", smt.ArraySort(smt.RealSort(), smt.IntSort()))
    N, m, n, k = smt.Ints("N m n k")
    eps = smt.Real("eps")
    is_cauchy = kd.define(
        "is_cauchy",
        [a],
        kd.QForAll(
            [eps],
            eps > 0,
            smt.Exists([N], kd.QForAll([m, k], m > N, k > N, abs(a[m] - a[k]) < eps)),
        ),
    )
    cauchy_mod = kd.define(
        "cauchy_mod",
        [a, mod],
        kd.QForAll(
            [eps],
            eps > 0,
            kd.QForAll([m, k], m > mod[eps], k > mod[eps], abs(a[m] - a[k]) < eps),
        ),
    )
    is_convergent = kd.define(
        "is_convergent",
        [a],
        kd.QForAll(
            [eps],
            eps > 0,
            smt.Exists([N], kd.QForAll([m], m > N, smt.Exists([x], abs(a[m] - x) < eps))),
        ),
    )
    seqsum = Function("seqsum", RSeq, R)
    # is_sum_convergent =

    # TODO. Should be less axioms
    # https://en.wikipedia.org/wiki/Limit_of_a_function
    delta, p, L = smt.Reals("delta p L")
    has_lim_at = kd.define(
        "has_lim_at",
        [f, p, L],
        kd.QForAll(
            [eps],
            eps > smt.RealVal(0),
            kd.QExists(
                [delta],
                delta > 0,
                kd.QForAll(
                    [x],
                    smt.RealVal(0) < abs(x - p),
                    abs(x - p) < delta,
                    abs(f[x] - L) < eps,
                ),
            ),
        ),
    )
    lim = smt.Function("lim", RFun, R, R)
    lim_def = kd.axiom(kd.QForAll([f, x, y], has_lim_at(f, x, y), lim(f, x) == y))

    has_diff_at = smt.Function("has_diff_at", RFun, R, R, smt.BoolSort())
    diff_at = kd.define("diff_at", [f, x], smt.Exists([y], has_diff_at(f, x, y)))
    cont_at = kd.define(
        "cont_at",
        [f, x],
        kd.QForAll(
            [eps],
            eps > 0,
            kd.QExists(
                [delta],
                delta > 0,
                kd.QForAll([y], abs(x - y) < delta, abs(f[x] - f[y]) < eps),
            ),
        ),
    )
    # smt.Function("cont_at", RFun, R, smt.BoolSort())

    is_diff = kd.define("is_diff", [f], smt.ForAll([x], diff_at(f, x)))
    is_cont = kd.define("is_cont", [f], smt.ForAll([x], cont_at(f, x)))
    diff_cont = kd.axiom(kd.QForAll([f], is_diff(f), is_cont(f)))

    sin_diff = kd.axiom(is_diff(sin))
    sin_cont = kd.prove(is_cont(sin), by=[sin_diff, diff_cont(sin)])


    # Since not all functions are differentiable, the connection of deriv to the analytic definition of derivative is a proof obligation
    deriv = smt.Function("deriv", RFun, RFun)
    deriv_const = kd.axiom(ForAll([x], deriv(const(x)) == const(0)))
    deriv_ident = kd.axiom(deriv(X) == const(1))
    deriv_sin = kd.axiom(deriv(sin) == cos)
    deriv_cos = kd.axiom(deriv(smt.Lambda([x], cos(x))) == smt.Lambda([x], -sin(x)))
    deriv_exp = kd.axiom(deriv(exp) == exp)
    deriv_add = kd.axiom(ForAll([f, g], deriv(f + g) == deriv(f) + deriv(g)))
    deriv_mul = kd.axiom(ForAll([f, g], deriv(f * g) == deriv(f) * g + f * deriv(g)))

    # Many notions of integrable.
    is_integ = smt.Function("is_integ", RFun, smt.BoolSort())

    Powser = kd.NewType("Powser", RSeq)
    # is_convergent_at

    # Bounds
    # https://arxiv.org/pdf/0708.3721
    # Marc Daumas, David Lester, and César Munoz. 2008. Verified real number calculations:
    # A library for interval arithmetic. IEEE Trans. Comput. 58, 2 (2008), 226–237.


    def sqrt_upper(x, n):
        assert n >= 0
        if n == 0:
            return x + 1
        else:
            y = sqrt_upper(x, n - 1)
            return (y + x / y) / 2


    def sqrt_lower(n, x):
        assert n >= 0
        return x / sqrt_upper(x, n)


    def sqrt_bnd(n):
        x = smt.Real("x")
        return kd.axiom(
            kd.QForAll(
                [x],
                x >= 0,
                smt.And(sqrt_lower(x, n) <= sqrt(x), sqrt(x) <= sqrt_upper(x, n)),
            )
        )


    def sin_lower(n, x):
        assert n >= 0
        sum(x**n)

%lprun -f foo foo()

Timer unit: 1e-09 s

Total time: 0.991412 s
File: /tmp/ipykernel_2188711/1481737934.py
Function: foo at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def foo():
     2         1       7755.0   7755.0      0.0      import kdrag.smt as smt
     3         1       4218.0   4218.0      0.0      from kdrag.smt import ForAll, Function
     4         1        932.0    932.0      0.0      import kdrag as kd
     5                                           
     6         1        330.0    330.0      0.0      """
     7                                               Definitions about the reals.
     8                                               Transcendental functions and bounds.
     9                                           
    10                                               """
    11         1      86484.0  86484.0      0.0      R = smt.RealSort()
    12         1     132792.0 132792.0      0.0      RFun = smt.Ar

In [1]:
from kdrag.all import *

https://stackoverflow.com/questions/29277650/solvability-of-nonlinear-real-arithmetic-with-premises

I could do randomized tuning to see if any of these speed up my typical query

induction (bool) enable generation of induction lemmas (default: false) Whaa?

In [26]:
x,y,z= smt.Reals("x y z")

def expand0(t):
    return z3.simplify(x*(y + z), som=True)

def expand(t):
    return kd.reflect.



x*y + x*z

In [28]:
z3.help_simplify()

algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
blast_select_store (bool) eagerly replace all (select (store ..) ..) term by an if-then-else term (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: f

In [24]:
s.help()


abce (bool) eliminate blocked clauses using asymmetric literals (default: false)
acce (bool) eliminate covered clauses using asymmetric added literals (default: false)
add_bound_lower (rational) (default: -2) lower bound to be added to unbounded variables.
add_bound_upper (rational) (default: 2) upper bound to be added to unbounded variables.
algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
anf (bool) enable ANF based simplification in-processing (default: false)
anf.delay (unsigned int) delay ANF simplification by in-processing round (default: 2)
anf.exlin (bool) enable extended linear simplification (default: false)
arith.auto_config_simplex (bool) force simplex solver in auto_config (default: false)
arith.bprop_on_pivoted_rows (bool) propagate bounds on rows changed by the pivot operation (default: true)
arith.branch_cut_ratio (unsigned int) branch/cut ratio for linear integer arithmetic (default: 2)
arith.dum

In [ ]:
import z3
x = z3.Real("x")
s = z3.Solver() 
s.set("")
s.add(z3.Not(z3.Implies(x >= 0, x**0.5 >= 0)))
s.check()
s.trail()

Z3Exception: b'cannot retrieve trail from solvers created using tactics'

In [21]:
import z3
p = z3.Bool("p")
x = z3.Real("x")
s = z3.Solver() 
s.set("unsat_core", True)
#s.add(p)
s.add(z3.Implies(p,z3.Not(z3.Implies(x >= 0, x**0.5 >= 0))))
s.check(p)
s.srep()
#s.unsat_core()


AttributeError: 'Solver' object has no attribute 'srepr'

In [14]:
import z3
x = z3.Real("x")
s = z3.Solver() 
s.assert_and_track(z3.Not(z3.Implies(x >= 0, x**0.5 >= 0)), "p")
s.check()
s.unsat_core()

[]

0.8 seconds. I am concerned. It isn't coming from a big spike. It's a lot of 2% here and there on proofs.
0.4 seconds if I use kd.kernel.prove
I could maybe use a les


In [2]:
import kdrag.theories.real as real 

In [2]:
x = smt.Real("x")
l = kd.Lemma(kd.QForAll([x], x >= 0, real.sqrt(real.sqr(x)) == x))
_x = l.fix()
l.unfold()
l.auto()
l.qed()


|- ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x))

In [40]:
_1 = kd.kernel.prove(smt.Implies(x >= 0, x**0.5 >= 0))

In [9]:
x = smt.Real("x")
l = kd.Lemma(
    kd.QForAll([x], x >= 0, real.sqrt(x) ** 2 == x)
)
_x = l.fix()
l.unfold()
l.auto()
l.qed()
#l.simp()
#kd.kernel.prove(l.top_goal().goal)

|- ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))

In [21]:
import kdrag.theories.algebra.group as grp
e = grp.e
x,y = smt.Consts("x y", grp.G)
l = kd.Lemma(kd.QForAll([x], kd.QForAll([y], x * y == y), x == e))
_x = l.fix()
l.intros()
l.eq(_x * e)
l.eq(_x, by=[grp.id_right(_x)])
id_unique2 = l.qed()

In [49]:
l = kd.Lemma(kd.QForAll([x], x >= 0, real.sqrt(x) >= 0))
_x = l.fix()
l.unfold()
l.auto(by=kd.kernel.prove(l.top_goal().goal))
l.qed()

LemmaError: ('prove', Implies(And, Implies(x!3005 >= 0, x!3005**(1/2) >= 0)), [], unknown)

In [42]:
smt.Implies(x >= 0, x**0.5 >= 0)

Implies(x >= 0, x**(1/2) >= 0)

In [33]:
import kdrag.hypothesis as hyp

hyp.nitpick(kd.QForAll([x], x >= 0, real.sqrt(x) >= 0), deadline=1000)

In [6]:
from kdrag.all import *
x,y,z = smt.Reals("x y z")
mul_assoc = kd.prove(
    smt.ForAll([x, y, z], real.mul(x, real.mul(y, z)) == real.mul(real.mul(x, y), z)), by=[real.mul.defn]
)

In [10]:
def simp_tac(e):
    trace = []
    e1 = kd.simp(e, trace=trace)
    return kd.kernel.prove(e == e1, by=trace)

simp_tac(real.mul(x, real.mul(y, z)))
simp_tac(

|- mul(x, mul(y, z)) == x*y*z

In [7]:
%%timeit
mul_assoc = kd.prove(
    smt.ForAll([x, y, z], real.mul(x, real.mul(y, z)) == real.mul(real.mul(x, y), z)), by=[kd.tactics.simp_tac(real.mul(x, real.mul(y, z))), ]
)

16.9 ms ± 252 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
l = kd.Lemma( smt.ForAll([x, y, z], real.mul(x, real.mul(y, z)) == real.mul(real.mul(x, y), z)))
_x,_y,_z = l.fixes()
l.unfold()
l.auto()
l.qed()

|- ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))

In [47]:
import kdrag.theories.real as real
import kdrag.reflect
from kdrag.all import *
import sympy
import sympy.abc
sympy_env = {**sympy.__dict__, **sympy.abc.__dict__}
def sympy_nbe(vs, e, globals={}):
    env = {**sympy.__dict__, **sympy.abc.__dict__, **globals}
    sympy_e = kd.utils.eval_(e, env)
    return sympy.lambdify([v.decl().name() for v in vs], sympy_e.simplify(), modules=[real])(*vs)


# add default parameter to convert tree to Basic.
def kd_sympify(e : smt.ExprRef, locals={}):
    return kd.reflect.eval_(e, sympy_env, locals={})
x = smt.Real("x")
#kd_sympyify(smt.Lambda([x], x*y*z))
#kd_sympyify(smt.Lambda([x], x*y*z), locals={"y": 3, "z": 4})
#kd.reflect.reify(smt.RealSort(), kd_sympyify(x*(x + 1)).expand())

def wrap(f):
    def res(e):
        se = kd_sympyify(e)
        svs = list(se.free_symbols)
        vs = [smt.Real(v.name) for v in svs]
        return sympy.lambdify(svs, f(se))(*vs)
    return res
expand = wrap(sympy.expand)
factor = wrap(sympy.factor)
simplify = wrap(sympy.simplify)
trigsimp = wrap(sympy.trigsimp)

def singularities(e):

def plot():
plot3d
plotimplicit


def series():

def limit():
def linsolve()

def solve(eqs, vs):
    se = [kd_sympyify(eq) for eq in eqs]
    svs = list(set().union(*[eq.free_symbols for eq in se]))
    vs = [smt.Real(v.name) for v in svs]
    return sympy.lambdify(svs, sympy.solve(se, vs), modules=[real])(*vs)

def subs(x, subst):
    return smt.substitute(x, *subst.items())

def diff():
    pass
def Sum():
    pass
def Integrate():
    pass
#sympy_nbe([x], 1 + 2 + real.sin(x)**2 + real.cos(x)**2)

# call simplify on all subterms. Maybe gives more clues. Do fixedpoint loop.
def simp_blast():


apart():
together():
collect():
dsolve
evalf
rewrite


combinatorics

x**2 + x

In [48]:
import fractions
sympy.abc.x + fractions.Fraction(1,3)

x + 1/3

sumswap(e, i,j):
    # find two nearby sums.
    # swap them as a theorem/

In [ ]:
Sum = kd.define("sum", [R, a, b], 
    smt.If(a >= b, 0, R(b) + Sum(R, a, b - 1))
)


sympy_env = {
    **sympy.__dict__,
    **sympy.abc.__dict__,
}

def sympy_eval(e):
    return kd.utils.eval_(e, globals=sympy_env)

def sympy_sum_of_sum(f, nab):
    sf = sympy_eval(f)
    n, a, b = nab
    sn,sa,sb = sympy_eval(n), sympy_eval(a), sympy_eval(b)
    res = sympy.Sum(sf, (n, a, b)).doit()
    if isinstance(res, sympy.Sum):
        raise ValueError("Sympy cannot evaluate the sum")
    else:
        res2 = kd.reflect.reify(smt.RealSort(), res)
        l = Lemma(Sum(Lambda([n],f)), a ,b) = res2)
        try:
            l.unfold(Sum) # maybe I don't even need to do this.
            _b = l.fix()
            l.induct(_b)
            l.simp() # x < 0
            l.simp() # x = 0
            l.simp() # x > 0
            return l.qed()
        except Exception as e:
            print(e)
            return l




sympy.Sum(f, (n, a, b))




# monotonic dict


In [ ]:
class MonotonicDict(dict):
    def __setitem__(self, key, value):
        v1 = self.get(key)
        if v1 is None:
            dict.__setitem__(self, key, value)
        else:
            raise ValueError("Cannot change value of existing key")


# better simp, best in pocket
Keep best in pocket for simp. Index of proof at which this happened.
Have a cutoff of current being 10x (a parameter) best size.

unfold limit
Hmm. This is annoying. Easier if I had a zipper


I dunno if this is a great idea.

In [ ]:
from typing import NamedTuple

class ZipperNode(NamedTuple):
    head : smt.FuncDeclRef
    left : list[smt.ExprRef] # arguments to the head function on the left
    right : list[smt.ExprRef] #arguments to the right

ExprZipper = list[ZipperNode]
class ExprZipper(NamedTuple):
    def __init__(self, e : smt.ExprRef):
        self..cursor = e
        self.zipper = []
    def arg(self, n):
        e = self.cursor
        children = e.children()
        self.zipper.append(ZipperNode(e.decl(), children[:n], children[n+1:]))
        self.cursor = children[n]
    def pop(self):
    def left(self):
    def right(self):
    def __iter__(self):


In [ ]:
zipper = [(e.decl(), e.children(), [])]
while zipper:
    (head, todo, done) = zipper.pop()
    if len(todo) == 0:
        if head in defns:
            smt.substitute(defn.body, *(zip(defn.args, done)))

        zipper[-1][1].append(head(*done))
        continue
    else:
        e = todo.pop()
        zipper.append(e.decl(), e.children(), [])
        






In [ ]:
def unfold(e: smt.ExprRef, decls=None, max_unfold=None, trace=None) -> smt.ExprRef:
    """
    Do a single unfold sweep, unfolding definitions defined by `kd.define`.
    The optional trace parameter will record proof along the way.
    `decls` is an optional list of declarations to unfold. If None, all definitions are unfolded.

    >>> x = smt.Int("x")
    >>> f = kd.define("f", [x], x + 42)
    >>> trace = []
    >>> unfold(f(1), trace=trace)
    1 + 42
    >>> trace
    [|- f(1) == 1 + 42]
    """
    if max_unfold == 0:
        return e
    if smt.is_app(e):
        decl = e.decl()
        children = []
        for c in e.children():
            if smt.is_app(c):
                n_unfolds += 1
        children = [unfold(c, decls=decls, trace=trace) for c in e.children()]
        defn = kd.kernel.defns.get(decl)
        if defn is not None and (decls is None or decl in decls):
            e1 = smt.substitute(defn.body, *zip(defn.args, children))
            e = e1
            if trace is not None:
                if isinstance(defn.ax.thm, smt.QuantifierRef):
                    trace.append((defn.ax(*children)))
                else:
                    trace.append(defn.ax)
            return e1
        else:
            return decl(*children)
    else:
        return e

# Reflection

I thought for a second if I knew the sort maybe I could just send over enough guys (each constructor) and use simp.
If there are any Ints that fails.

I could symexec.

Replace refernce to current function to refer to symbolic self?

Put into __wrapped__ field.
Hypothesis test that the two agree.




# bitvec n

# serialize


In [3]:
from kdrag.all import *
import kdrag.theories.nat as nat

import json
def expr_default(x):
    if isinstance(x, smt.ExprRef):
        return x.serialize()
    else:
        raise TypeError(f"Cannot serialize {x}")
json.dumps(nat.add_Z, default=expr_default)


'["(declare-datatypes ((Nat 0)) (((Z) (S (pred Nat)))))\\n(declare-fun F (Bool) Bool)\\n(declare-fun add (Nat Nat) Nat)\\n(assert (F (forall ((x Nat)) (= (add Z x) x))))\\n", [["(declare-datatypes ((Nat 0)) (((Z) (S (pred Nat)))))\\n(declare-fun F (Bool) Bool)\\n(declare-fun add (Nat Nat) Nat)\\n(assert (F (forall ((x Nat) (y Nat))\\n     (let ((a!1 (ite ((_ is Z) x) y (S (add (pred x) y)))))\\n       (= (add x y) a!1)))))\\n", ["d", "e", "f", "i", "n", "i", "t", "i", "o", "n"], false]], false]'

# Seperation

Everything is contingent upon the current valid heap addresses.




In [ ]:

SBool = smt.ArraySort(smt.SetSort(smt.IntSort()), smt.BoolSort())

valid = smt.Const("valid", smt.SetSort(smt.IntSort()))
def pto(heap, x, y):
    return Lambda([valid], smt.And(valid[x],  heap[x] == y))

valid1, valid2 = smt.Consts("valid1 valid2", smt.SetSort(smt.IntSort()))
def sep(p, q):
    return Lambda([valid], smt.SetUnion(valid1,valid2) == valid, smt.SetInterSection(valid1,valid2) == EmptySet,  
                  p[valid1], q[valid2] )



# TLA
https://lamport.azurewebsites.net/pubs/lamport-actions.pdf lamport actions paper


Hmm. Note that propsitions are Set(IntSort)
Since diff, union, and, etc kind of have built in meaning 


In [86]:


def TBools(names):
    return smt.Consts(names, smt.SetSort(smt.IntSort()))

def TInts(names):
    return smt.Consts(names, smt.SetSort(smt.IntSort()))

t = smt.FreshInt(prefix="t")
kd.notation.add.register(smt.ArraySort(smt.IntSort(), smt.IntSort()), lambda x,y: smt.Lambda([t], x[t] + y[t]))

#kd.notation.rem.register(smt.ArraySort(smt.IntSort(), smt.IntSort()), lambda x,y: smt.Lambda([t], x[t] % y[t]))


def TAnd(*args):
    return functools.reduce(smt.SetIntersect, args)

def TOr(*args):
    return functools.reduce(smt.SetUnion, args)

def Next(f : smt.DatatypeSortRef):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], f[t + 1])

def Box(p, unchanged=None):
    t,t1= smt.FreshInt(prefix="t"), smt.FreshInt(prefix="t")
    q  = smt.Lambda([t], kd.QForAll([t1], t1 >= t, p[t1]))
    if unchanged is None:
        return q
    else:
        q  = [q]
        q.extend(map(UNCHANGED, unchanged))
        return TAnd(*q)

def Diamond(p):
    t,t1= smt.FreshInt(prefix="t"), smt.FreshInt(prefix="t")
    return smt.Lambda([t], kd.QExists([t1], t1 >= t, p[t1]))

def TIntVal(n):
    return smt.K(smt.IntSort(), smt.IntVal(n))

def TIf(c, t, f):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], smt.If(c[t], t[t], f[t]))

def TNot(e):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], smt.Not(e[t]))

def TImplies(e1, e2):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], smt.Implies(e1[t], e2[t]))

def UNCHANGED(p):
    return TEq(Next(p), p)

def TEq(e1, e2):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], e1[t] == e2[t])

def TInts(names):
    return smt.Consts(names, smt.ArraySort(smt.IntSort(), smt.IntSort()))

hr,min = TInts("hr min")

vars = [hr]


zero = TIntVal(0)
one = TIntVal(1)
twelve = TIntVal(12)
Init = TEq(hr, one)#, TEq(min, zero))

#Step = TIf(TEq(hr, twelve), TEq(Next(hr), one), TEq(Next(hr), hr + one)) #, TEq(Next(min), min))
TEq(Next(hr), hr + one)

Lambda(t!657,
       Lambda(t!656, hr[t!656 + 1])[t!657] ==
       Lambda(t!654, hr[t!654] + K(Int, 1)[t!654])[t!657])

In [ ]:

Spec = TAnd(Init, Box(Step,unchanged=vars))

def TAssert(p):
    return p[0]

smt.simplify(TAssert(Init))

smt.simplify(TAssert(Spec))


def bmc(p, k):
    # traverse p, instantiate all quantifiers over t to 0..k
    p = smt.simplify(p)
    if isinstance(p, smt.QuantifierRef):
        if p.is_forall() and p.num_vars() == 1 and p.var_sort(0) == smt.IntSort():
            vs, body = kd.utils.open_binder(p)
            body1 = bmc(body, k)
            return smt.And([smt.substitute(bmc(body, k), (vs[0], smt.IntVal(i))) for i in range(k)])
        elif p.is_lambda():
            vs,body = kd.utils.open_binder(p)
            return smt.Lambda(vs, bmc(body, k))
        else:
            raise NotImplementedError("Cannot handle quantifier")
    elif smt.is_app(p):
        return p.decl()(*[bmc(c, k) for c in p.children()])
    else:
        raise NotImplementedError("Cannot handle expression", p)

#bmc(TAssert(Spec), 2)
smt.simplify(TAssert(Spec))
smt.simplify(bmc(TAssert(Spec), 3))

VARIABLE hr

vars == << hr >>

Init == (* Global variables *)
        /\ hr = 1

Next == IF hr = 12
           THEN /\ hr' = 1
           ELSE /\ hr' = hr + 1

Spec == Init /\ [][Next]_vars

In [57]:
x = smt.IntVal(7)
x.decl()


[]

In [1]:
from kdrag.all import *


def TBoolSort():
    return smt.Array(smt.IntSort(), smt.BoolSort())

NowNext = kd.Enum("NowNext", "Now Next")

def ABoolSort():
    return smt.Array(NowNext, smt.BoolSort())

def ABools(xs):
    return smt.Consts(xs, ABoolSort())

next = kd.define("next", [x], Lambda([n], smt.If(n == NowNext.Next, x, smt.Const("undefined", smt.BoolSort()))))


Box = kd.function("Box", [ARel], TRel)



In [6]:
import time
now = time.time()

In [7]:
time.time() - now

6.298983335494995